# Create summary tables in Redshift

Replaces or creates a new view in Redshift with a query built across tables. Can be run at regular intervals or after data loads in order to refresh data models.

In [ ]:
import psycopg2
import psycopg2.extras

In [ ]:
conn = stitch_context.connections['Default Warehouse']['client']
cur = conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)

In [ ]:
# build command to create a new table
# use CREATE OR REPLACE VIEW name AS in order to avoid conflicts when trying to replace data
# https://docs.aws.amazon.com/redshift/latest/dg/r_CREATE_VIEW.html

create_table = """
create or replace view build_summary_table as (
  with a as (
    select
      cid as primary_key
    from
      schema_a.table_a
    limit 100),
  b as (
    select
      client_id as foreign_key,
      count(*) as total
    from
      schema_b.table_b
    group by
      1
    order by
      2 desc),
    c as (
      select
        a.*,
        nvl (b.total,
          0) as total
      from
        a
      left join b on a.primary_key = b.foreign_key
)
  select
    total,
    count(*)
  from
    c
  group by
    1
);
"""

In [ ]:
# execute command to create a new table
try:
    cur.execute(create_table)
    log.info('Successfully executed CREATE TABLE AS command')

# make the changes to the database persistent
    conn.commit()
    log.info('Successfully committed changes to the database')

finally:
    cur.close()
    conn.close()